# Kuidas luua AI agente püsiva mäluga, kasutades Cognee't

See märkmik näitab, kuidas luua intelligentseid AI agente, kellel on arenenud mälufunktsioonid, kasutades [**cognee**](https://www.cognee.ai/) - avatud lähtekoodiga AI mälu, mis ühendab teadmiste graafikud, semantilise otsingu ja sessioonihalduse, et luua kontekstitundlikke AI süsteeme.

## 🎯 Õpieesmärgid

Selle õpetuse lõpuks mõistad, kuidas:
- **Luua teadmiste graafikuid, mis põhinevad sisseehitatud mudelitel**: Muuta struktureerimata tekst struktureeritud ja päringuteks sobivaks teadmisteks
- **Rakendada sessioonimälu**: Luua mitme pöördega vestlusi automaatse konteksti säilitamisega
- **Salvestada vestlusi**: Valikuliselt talletada olulisi interaktsioone pikaajalisse mällu tulevaseks kasutamiseks
- **Pärida loomuliku keele abil**: Juurdepääs ja ajaloolise konteksti kasutamine uutes vestlustes
- **Visualiseerida mälu**: Uurida oma agendi teadmiste graafiku seoseid


## 🏗️ Mida Sa Lood

Selles juhendis loome **Koodimisassistendi** püsiva mäluga, mis:

### 1. **Teadmistebaasi loomine**
   - Kogub arendajate profiili ja ekspertteadmiste infot
   - Töötleb Python programmeerimise põhimõtteid ja parimaid praktikaid
   - Salvestab ajaloolisi vestlusi arendajate ja AI assistentide vahel

### 2. **Sessiooniteadlikud vestlused**
   - Säilitab konteksti mitme küsimuse vahel samas sessioonis
   - Salvestab automaatselt iga küsimuse/vastuse paari tõhusaks otsinguks
   - Pakub sidusaid ja kontekstipõhiseid vastuseid vestluse ajaloo põhjal

### 3. **Pikaajaline mälu**
   - Salvestab olulised vestlused pikaajalisse mällu
   - Otsib asjakohaseid mälestusi teadmistebaasist ja varasematest sessioonidest, et rikastada uusi suhtlusi
   - Loob kasvava teadmistebaasi, mis aja jooksul paraneb

### 4. **Tark mälestuste otsing**
   - Kasutab graafikupõhist semantilist otsingut, et leida asjakohast infot kogu salvestatud teadmiste hulgast
   - Filtreerib otsinguid andmealamrühmade järgi (arendaja info vs. põhimõtted)
   - Kombineerib mitmeid andmeallikaid, et pakkuda põhjalikke vastuseid


## 📋 Eeltingimused ja Seadistamine

### Süsteeminõuded

Enne alustamist veendu, et sul on:

1. **Python Keskkond**
   - Python 3.9 või uuem
   - Virtuaalne keskkond (soovitatav)
   
2. **Redis Vahemälu** (Nõutav sessioonihalduseks)
   - Kohalik Redis: `docker run -d -p 6379:6379 redis`
   - Või kasuta hallatud Redis-teenust
   
3. **LLM API Juurdepääs**
   - OpenAI API võti või muud teenusepakkujad (vaata [dokumentatsiooni](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Andmebaasi Konfiguratsioon**
   - Vaikimisi pole konfiguratsiooni vaja. Cognee kasutab failipõhiseid andmebaase (LanceDB ja Kuzu)
   - Soovi korral saad seadistada Azure AI Search vektoripoena (vaata [dokumentatsiooni](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Keskkonna Konfiguratsioon

Loo oma projekti kausta `.env` fail järgmiste muutujatega:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Cognee mäluarhitektuuri mõistmine

### Kuidas Cognee töötab

Cognee pakub keerukat mälusüsteemi, mis ületab lihtsa võtme-väärtuse salvestuse:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Põhikomponendid:

1. **Teadmusgraafik**: Salvestab üksused, seosed ja semantilised ühendused
2. **Vektorembeddingud**: Võimaldab semantilist otsingut kogu salvestatud teabe ulatuses
3. **Seansi vahemälu**: Säilitab vestluse konteksti seansi sees ja vahel
4. **NodeSets**: Korraldab andmed loogilistesse kategooriatesse sihipäraseks otsinguks

### Mälutüübid selles juhendis:

- **Püsiv mälu**: Pikaajaline salvestus teadmusgraafikus
- **Seansi mälu**: Ajutine vestluse kontekst Redis-vahemälus
- **Semantiline mälu**: Vektoripõhine sarnasuse otsing kogu andmete ulatuses


## 📦 Paigalda vajalikud paketid

Paigalda Cognee koos Redis toe abil sessioonihalduseks:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Keskkonna algatamine ja teekide laadimine

Veendu, et:
1. Redis töötab (näiteks Dockeriga: `docker run -d -p 6379:6379 redis`)
2. Keskkonnamuutujad on seadistatud enne vahemälu moodulite importimist
3. Vajadusel taaskäivita kernel ja käivita lahtrid järjekorras

Järgmine lahter teeb järgmist:
1. Laadib keskkonnamuutujad `.env` failist
2. Konfigureerib Cognee sinu LLM seadistustega
3. Lubab vahemälu sessioonihalduseks
4. Kontrollib, et kõik komponendid oleksid korralikult ühendatud


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Konfigureeri salvestuskataloogid

Cognee kasutab oma toimingute jaoks kahte eraldi kataloogi:
- **Andmete juurkataloog**: Salvestab sisestatud dokumendid ja töödeldud andmed
- **Süsteemi juurkataloog**: Sisaldab teadmiste graafi andmebaasi ja süsteemi metaandmeid

Selles juhendis loome eraldatud kataloogid järgmiselt:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Mälu oleku lähtestamine

Enne kui alustame oma mälusüsteemi loomist, veendume, et alustame puhtalt lehelt.

> 💡 **Näpunäide**: Võite selle sammu vahele jätta, kui soovite säilitada olemasolevaid mälestusi oma varasematest käivitustest, kui kasutate seda märkmikku hiljem.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Osa 1: Teadmistebaasi loomine

### Andmeallikad meie arendaja assistendi jaoks

Teadmistebaasi loomiseks kasutame kolme tüüpi andmeid:

1. **Arendaja profiil**: Isiklikud teadmised ja tehniline taust
2. **Python'i parimad praktikad**: Python'i Zen koos praktiliste juhistega
3. **Ajaloolised vestlused**: Varasemad küsimuste ja vastuste sessioonid arendajate ja AI assistentide vahel

See mitmekesine andmestik võimaldab meie agendil:
- Mõista kasutaja tehnilist konteksti
- Rakendada parimaid praktikaid soovitustes
- Õppida varasematest edukatest interaktsioonidest


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Töötle andmed teadmiste graafikuks

Nüüd muudame oma toorteksti struktureeritud mäluks. See protsess:

1. **Lisab andmed NodeSets'i**: Korraldab teabe loogilistesse kategooriatesse
   - `developer_data`: Arendaja profiil ja vestlused
   - `principles_data`: Python'i parimad praktikad ja juhised

2. **Käivitab Cognify Pipeline'i**: Ekstraheerib üksused, seosed ja loob sisukorrad
   - Tuvastab peamised mõisted
   - Loob semantilised ühendused seotud teabe vahel
   - Genereerib vektorite sisukorrad

See võib võtta veidi aega, kuna LLM töötleb teksti ja ehitab graafiku struktuuri:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualiseeri teadmiste graafik

Uurime oma teadmiste graafiku struktuuri. Visualisatsioon näitab:
- **Sõlmed**: Tekstist tuvastatud üksused (mõisted, tehnoloogiad, inimesed)
- **Servad**: Suhted ja seosed üksuste vahel
- **Klastrid**: Semantilise sarnasuse alusel grupeeritud seotud mõisted

Avage loodud HTML-fail oma brauseris, et graafikut interaktiivselt uurida:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Rikasta mälu Memify abil

Funktsioon `memify()` analüüsib teadmiste graafikut ja loob andmete kohta nutikaid reegleid. See protsess:
- Tuvastab mustrid ja parimad praktikad
- Loob sisu põhjal rakendatavad juhised
- Määratleb seosed erinevate teadmiste valdkondade vahel

Need reeglid aitavad agendil teha teadlikumaid otsuseid küsimustele vastamisel. Teise visualiseerimise jäädvustamine võimaldab võrrelda, kuidas graafik rikastamise järel tihedamaks muutub.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Osa 2: Nutikas mäluotsing

### Demonstratsioon 1: Dokumentidevaheline teadmiste integreerimine

Nüüd, kui meie teadmiste graafik on loodud, testime, kuidas Cognee ühendab teavet mitmest allikast, et vastata keerukatele küsimustele.

Esimene päring näitab:
- **Semantiline mõistmine**: Leidmine asjakohastele mõistetele isegi siis, kui neid pole otseselt mainitud
- **Ristviitamine**: Arendaja profiili ühendamine Python'i põhimõtetega
- **Kontekstuaalne arutlemine**: Parimate praktikate rakendamine konkreetsetele lahendustele

### Demonstratsioon 2: Filtreeritud otsing NodeSets'iga

Teine päring näitab, kuidas suunata otsing konkreetsetele teadmiste graafiku alamhulkadele:
- Kasutab `node_name` parameetrit, et otsida ainult `principles_data` piires
- Pakub keskendunud vastuseid konkreetsest teadmiste valdkonnast
- Kasulik, kui vajate valdkonnaspetsiifilist teavet


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Osa 3: Sessioonihalduse seadistamine

### Vestlusmälu lubamine

Sessioonihaldus on oluline, et säilitada konteksti mitme suhtluse jooksul. Siin teeme järgmist:

1. **Kasutajakonteksti initsialiseerimine**: Loome või hangime kasutajaprofiili sessiooni jälgimiseks
2. **Vahemootori seadistamine**: Ühendame Redis'ega, et salvestada vestluste ajalugu
3. **Sessioonimuutujate lubamine**: Seadistame kontekstimuutujad, mis püsivad päringute vahel

> ⚠️ **Oluline**: Selleks peab Redis töötama ja keskkonnas peab olema `CACHING=true`


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Abifunktsioon: Vaata sessiooni ajalugu

See abifunktsioon võimaldab meil vaadata Redis'is salvestatud vestluste ajalugu. See on kasulik:
- Sessioonihalduse silumiseks
- Vestluste vahemällu salvestamise kontrollimiseks
- Mõistmaks, milline kontekst on agendile kättesaadav


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sessie 1: Asünkroonse toe labor — Esimene küsimus

Alustage `async-support-lab` sessiooni, küsides telemeetria-sõbralikke asyncio mustreid massiivse veebikraapija jaoks. Graafik on juba teadlik asyncio, aiohttp ja monitooringu praktikate kohta, seega peaks vastus peegeldama varasemaid vestlusi, kohandades samal ajal vastust uuele küsimusele.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Kontrolli sessiooni 1 mälu pärast esimest vahetust

Käivitades `show_history(session_1)` kohe pärast esimest küsimust, kinnitab see, et Cognee salvestas nii küsimuse kui ka vastuse Redis andmebaasi. Sa peaksid nägema ühte kirjet koos samaaegse juhendamisega.


In [ ]:
await show_history(session_1)

## Sessioon 1: Andmemudelite järelarutelu

Järgmisena küsime: "Millal peaksin valima dataclasses versus Pydantic?" kasutades sama sessiooni ID-d. Cognee peaks ühendama Python'i põhimõtted ja varasemad FastAPI vestlused, et pakkuda nüansirikast nõu—näidates, et kontekst kandub edasi nimetatud sessiooni raames.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Kinnita, et sessiooni 1 ajalugu sisaldab mõlemat vahetust

Veel üks `show_history(session_1)` käsk peaks näitama kahte küsimuse-vastuse kirjet. See vastab Mem0 labori "mälutaasesituse" sammule ja tõestab, et lisavahetused pikendavad sama transkriptsiooni.


In [ ]:
await show_history(session_1)

## Sessioon 2: Kujunduse ülevaate teema — Uus sessioon

Et näidata teemade eraldatust, käivitame `design-review-session` ja küsime logimise juhiseid juhtumite ülevaatamiseks. Kuigi aluseks olev teadmistebaas on sama, hoiab uus sessiooni ID ärakirjad eraldi.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Ülevaatuse Sessioon 2 Ajalugu

`show_history(session_2)` peaks kuvama ainult disainiülevaate küsimuse/vastuse paari. Võrdle seda Sessiooniga 1, et näidata, kuidas Cognee hoiab eraldiseisvaid transkripte, samal ajal jagatud teadmistegraafikut uuesti kasutades.


In [ ]:
await show_history(session_2)

## Kokkuvõte

Palju õnne! Olete just andnud oma kodeerimisabilisele tõelise pikaajalise mälukihi, mida toetab Cognee.

Selles juhendis võtsite toorandmed (kood, dokumendid, vestlused) ja muutsid need graafikuks + vektormäluks, mida teie agent saab otsida, analüüsida ja pidevalt täiustada.

Mida olete õppinud

1. **Toortekstist tehisintellekti mäluni**: Kuidas Cognee töötleb struktureerimata andmeid ja muudab need intelligentseks, otsitavaks mäluks, kasutades kombineeritud vektori + teadmiste graafi arhitektuuri.

2. **Graafi rikastamine memify abil**: Kuidas minna kaugemale lihtsast graafi loomisest ja kasutada memify't, et lisada tuletatud fakte ja rikkamaid seoseid olemasolevale graafile.

3. **Mitmed otsingustrateegiad**: Kuidas pärida mälu erinevate otsingutüüpidega (graafiteadlik Q&A, RAG-stiilis lõpetamine, ülevaated, toorfragmendid, koodiotsing jne), sõltuvalt sellest, mida teie agent vajab.

4. **Visuaalne uurimine**: Kuidas inspekteerida ja siluda seda, mida Cognee on loonud, kasutades graafikavisualiseeringuid ja Cognee kasutajaliidest, et saaksite tegelikult näha, kuidas teadmised on struktureeritud.

5. **Sessiooniteadlik mälu**: Kuidas kombineerida sessioonipõhist konteksti püsiva semantilise mäluga, et agendid saaksid mäletada erinevate sessioonide vahel ilma kasutajatevahelist teavet lekkimata.


## Peamised järeldused
1. Mälu kui teadmiste graafik, mida toetavad manused

    - **Struktureeritud arusaam**: Cognee ühendab vektorpõhise andmebaasi ja graafipõhise andmebaasi, nii et teie andmed on otsitavad nii tähenduse kui ka seoste järgi. Cognee kasutab vaikimisi failipõhiseid andmebaase (LanceDB vektorite jaoks, Kuzu graafipõhise andmebaasi jaoks).

    - **Seostest teadlik otsing**: Vastused põhinevad mitte ainult "sarnasel tekstil", vaid ka sellel, kuidas üksused omavahel seotud on.

    - **Elav mälu**: Mälukiht areneb, kasvab ja jääb päringuteks kättesaadavaks kui üks ühendatud graafik.

2. Otsingu- ja põhjendusrežiimid
    - **Hübriidotsing**: otsing ühendab vektori sarnasuse, graafistruktuuri ja LLM-i põhjenduse, alates toorandmete otsimisest kuni graafikuteadliku küsimustele vastamiseni.

    - **Sobita režiim ülesandega**: Kasuta loomuliku keele vastuste jaoks täitmisstiilis režiime ning toorandmete, kokkuvõtete või graafikute jaoks režiime, kui agent vajab konteksti või oma põhjenduste juhtimist.

3. Isikupärastatud, sessiooniteadlikud agendid
    - **Sessioonikontekst + pikaajaline mälu**: Cognee hoiab lühiajalise "teema" konteksti eraldi pikaajalisest, kasutaja- või organisatsioonitasemel mälust.

## Reaalsed rakendused

1. **Vertikaalsed tehisintellekti agendid**

    Kasutage selle märkmiku mustrit, et luua valdkonnaspetsiifilisi kaaslasi, kes tuginevad Cognee'le kui nende otsingu- ja põhjendustuumale:

- **Arendajate kaaslased**: Koodi ülevaatus, intsidentide analüüs ja arhitektuuri assistendid, kes läbivad koodi, API-sid, disainidokumente ja pileteid kui ühtset mälugraafikut.

- **Kliendisuunalised kaaslased**: Tugiteenuste või kliendiedu agendid, kes ammutavad teavet tootetokumentidest, KKK-dest, CRM-i märkmetest ja varasematest piletitest graafikuteadliku otsingu ja viidatud vastustega.

- **Sisemised eksperdi kaaslased**: Poliitika, juriidika või turvalisuse assistendid, kes põhjendavad omavahel seotud reeglite, juhiste ja ajalooliste otsuste põhjal, mitte eraldiseisvate PDF-ide alusel.

    Cognee on selgelt positsioneeritud kui püsiv ja täpne mälu tehisintellekti agentidele, pakkudes elavat teadmiste graafikut, mis asendab agentide taga ad-hoc vektoripoodide ja kohandatud graafikukoodi kombinatsioone.

2. **Andmesilode ühendamine üheks mäluks**

    Sama lähenemine aitab teil luua ühtse mälukihi hajutatud allikate vahel:

- **Silodest üheks graafikuks**: Importige struktureeritud (nt andmebaasid) ja struktureerimata andmed (nt dokumendid, vestlused) ühte graafikusse, mida toetavad manused, selle asemel et luua iga süsteemi jaoks eraldi indeksid.

- **Allikateülene põhjendus viidetega**: Tehke mitmeastmeline põhjendus kõige üle—"ühendage" logid, mõõdikud ja dokumendid graafiku kaudu—ja tagastage siiski põhjendatud vastused koos allikaviidetega.

- **Teadmiste keskused**: Valdkondades nagu pangandus või haridus kasutatakse Cognee't juba PDF-ide, sisemiste süsteemide ja rakenduste andmete ühendamiseks ühte teadmiste graafikusse, et agendid saaksid vastata küsimustele täpse ja viidatud kontekstiga.

## Järgmised sammud

Olete rakendanud mälusilmuse põhiosa. Siin on loomulikud laiendused, mida saate ise proovida (vt [Cognee dokumentatsiooni](https://docs.cognee.ai/) üksikasjade jaoks):

1. **Katsetage ajalist teadlikkust**: Lülitage sisse ajaline kognitsioon, et eraldada tekstist sündmusi ja ajatempleid.

2. **Tutvustage ontoloogiapõhist põhjendust**: Määratlege oma valdkonna jaoks OWL-ontoloogia. Kasutage Cognee ontoloogia tuge, et eraldatud üksused ja seosed oleksid seotud selle skeemiga, parandades graafiku kvaliteeti ja valdkonnaspetsiifilisi vastuseid.

3. **Lisage tagasisideahel**: Laske Cognee'l kohandada graafiku servade kaale reaalsete kasutajate tagasiside põhjal, nii et otsing paraneb aja jooksul, mitte ei jää staatiliseks.

4. **Häälestage isikupärastamise ja sessioonikäitumise jaoks**: Kasutage kasutaja ID-sid, rentnikke ja andmekogumeid, et anda igale inimesele või meeskonnale oma vaade jagatud mälumootorile.

5. **Laiendage keerukamate agentide jaoks**: Ühendage Cognee agentide raamistikesse, et luua mitmeagendilisi süsteeme, mis kõik jagavad sama mälukihti. *Microsoft Agent Framework x Cognee plugin on peagi tulemas.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palun arvestage, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul on soovitatav kasutada professionaalset inimtõlget. Me ei vastuta arusaamatuste või valesti tõlgendamise eest, mis tulenevad selle tõlke kasutamisest.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
